# 1 Задание
У нас есть кусочек исследования, который мы теперь хотим поставить на регулярное выполнение. Пожалуйста, перепеши код, отформатировав его для удобства дальнейшего переиспользования и исправив ошибки, если они есть.
Напиши саммари по тому, что ты изменил в коде и почему.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import time
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patheffects as path_effects
from ibis import schema
import warnings
warnings.filterwarnings("ignore")

today_dt = (datetime.today()).date()

#today_dt = today_dt - timedelta(days=1)
today = today_dt.strftime('%Y-%m-%d')
yesterday_dt = today_dt - timedelta(days=1)
yesterday = yesterday_dt.strftime('%Y-%m-%d')

two_days_ago_start = (today_dt - timedelta(days=2)).strftime('%Y-%m-%d')

week_ago_start = (today_dt - timedelta(days=7)).strftime('%Y-%m-%d')
two_weeks_ago_start = (today_dt - timedelta(days=14)).strftime('%Y-%m-%d')
yesterday, week_ago_start, two_weeks_ago_start

last_week_day = (today_dt - timedelta(days=8)).strftime('%Y-%m-%d')
last_year_day = (yesterday_dt - relativedelta(years=1)).strftime('%Y-%m-%d')


total_aviasales_bookings = session.query( f"""
    select  to_date(booked_at) as period
, count(booking_id) as total_bookings_aviasales
from bd.bookings
where to_date(booked_at)  >= '{two_weeks_ago_start}'
                and to_date(booked_at)  <= '{yesterday}'
group by 1
order by 1
""")

total_aviasales_gross_profit = session.query( f"""
     select  to_date(booked_at) as period
, sum(profit) as total_profit_aviasales

from bd.bookings
where to_date(booked_at)  >= '{two_weeks_ago_start}'
                and to_date(booked_at)  <= '{yesterday}'
group by 1
order by 1
""")
df_bookings_sender = session.query(f"""

        select booked_at
    , count(if(sender_id in (10, 20, 30), booking_id, Null)) as bookings_sender
    , count(booking_id) as bookings
    , count(if(sender_id in (10, 20, 30), booking_id, Null)) * 100/ count(booking_id) as share_of_sender_bookings
    from bd.bookings
    where to_date(booked_at)  >= '{two_weeks_ago_start}'
                and to_date(booked_at)  <= '{yesterday}'
    group by 1
    order by 1
            """
            )
df_bookings_sender_last_year = session.query(f"""

        select booked_at
    , count(if(sender_id in (10, 20, 30), booking_id, Null)) as bookings_sender
    , count(booking_id) as bookings
    , count(if(sender_id in (10, 20, 30), booking_id, Null)) * 100/ count(booking_id) as share_of_sender_bookings
    from bd.bookings
    where to_date(booked_at)  >= '{last_year_day}'
                and to_date(booked_at)  <= '{yesterday}'
    group by 1
    order by 1
            """
            )

data_copy = total_aviasales_gross_profit.copy()
data_copy['total_profit_aviasales_forc'] = data_copy.apply(lambda row: row['total_profit_aviasales'] * 2 + np.sqrt(row['total_profit_aviasales']), axis=1)

df_yesterday = data_copy.copy()
df_yesterday['period'] = (pd.to_datetime(df_yesterday['period']) - timedelta(days=1)).dt.strftime('%Y-%m-%d')

df_merged = data_copy.merge(df_yesterday, left_on='period', right_on='period', how='left')

fig, axs = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Графики', fontsize=16)

axs[0, 0].plot(total_aviasales_bookings['period'], total_aviasales_bookings['total_bookings_aviasales'], marker='o')
axs[0, 0].set_title('Total Bookings Aviasales')
axs[0, 0].set_xlabel('Date')
axs[0, 0].set_ylabel('Total Bookings')

axs[0, 1].plot(total_aviasales_gross_profit['period'], total_aviasales_gross_profit['total_profit_aviasales'], marker='o')
axs[0, 1].set_title('Total Profit Aviasales')
axs[0, 1].set_xlabel('Date')
axs[0, 1].set_ylabel('Total Profit')

axs[1, 0].plot(df_bookings_sender['pdate'], df_bookings_mego['bookings'], marker='o')
axs[1, 0].set_title('Sender Bookings')
axs[1, 0].set_xlabel('Date')
axs[1, 0].set_ylabel('Bookings')

axs[1, 1].plot(df_bookings_sender_last_year['pdate'], df_bookings_mego_last_year['bookings'], marker='o')
axs[1, 1].set_title('Sender Bookings year')
axs[1, 1].set_xlabel('Date')
axs[1, 1].set_ylabel('Bookings')


for ax in axs.flat:
    if ax.get_title() == 'Total Bookings Aviasales':
        ax.plot(df_merged['period'].iloc[-1], total_aviasales_bookings[total_aviasales_bookings['period'] == df_merged['period'].iloc[-1]]['total_bookings_aviasales'], marker='o', markersize=8, label='Last Date', color='red', markeredgewidth=2)
    elif ax.get_title() == 'Total Profit Aviasales':
        ax.plot(df_merged['period'].iloc[-1], total_aviasales_gross_profit[total_aviasales_gross_profit['period'] == df_merged['period'].iloc[-1]]['total_profit_aviasales'], marker='o', markersize=8, label='Last Date', color='red', markeredgewidth=2)
    elif ax.get_title() == 'Sender Bookings':
        ax.plot(df_merged['period'].iloc[-1], df_bookings_sender[df_bookings_sender['pdate'] == df_merged['period'].iloc[-1]]['bookings'], marker='o', markersize=8, label='Last Date', color='red', markeredgewidth=2)
    elif ax.get_title() == 'Sender Bookings':
        ax.plot(df_merged['period'].iloc[-1], df_bookings_sender_last_year[df_bookings_sender_last_year['pdate'] == df_merged['period'].iloc[-1]]['bookings'], marker='o', markersize=8, label='Last Date', color='red', markeredgewidth=2)

plt.show()

# 2 Задание
В поисковую выдачу билетов была разработана и внедрена скоринговая модель, ранжирующая билеты.
Ее цель - помочь пользователю выбрать лучший авиабилет самым быстрым способом, подняв его в топ выдачи.

Пожалуйста, внимательно изучи поисковую выдачу сайта и предложи:

Какими метриками бы ты отслеживал качество работы модели?

Какие визуализации могли бы помочь быстро подсветить проблему и разобраться менеджерам, что пошло не так?

Что важно учесть и какие срезы стоит смотреть отдельно?

Какие данные тебе нужны, чтобы все это сделать?


Для выполнения задания ты можешь самостоятельно сгенерировать данные с учетом ожидаемых распределений или просто описать ход своей работы.